In [13]:
import numpy as np
import geopandas as gpd
import pandas as pd
import folium
import mapclassify
from folium import plugins
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from folium import IFrame

In [15]:
def conf_defaults():
    # Define some variables for later use
    conf = {
        'center_lat': 20,  # Latitude of the center of the map
        'center_lon': -103,  # Longitude of the center of the map
        'map_zoom': 8,  # Zoom factor of the map
        'ID': 'PFAF_ID', # Common field Geojson and Data in CSV
        'csv': 'agave_riskscores.csv', # Data
        'geo' : 'AGAVE.geojson', #geojson file
        'x': 'Quantity_Class', # Variable x 
        'y': 'land_theme_risk',  # Variable y
        'legend_x' : 'variable x',
        'legend_y' : 'variable y',
        'id': 'Quantity_Rank_ABC', # Catchment name / polygon name
        'bmode': 'equal intervals', # Select percentiles, equal intervals, custom, ficher-jenks, natural breaks, HeadTailBreaks
        'custom_breaks_x': [0, 100, 200, 300, 400], #if custom
        'custom_breaks_y': [0, 100, 200, 300, 400], #if custom
        'custom_breaks' : [0,1,2,3],
        'choropleth' : 'pink-blue' #from color_sets
        }
    return conf

In [17]:
conf = conf_defaults()

In [19]:
color_sets = {
    'pink-blue':   ["#e8e8e8", "#c8e1e1", "#a6d9d9", "#81d1d1", "#5ac8c8", "#dec8d9", "#c8c8d9", "#a6c8d9", "#81c8d1", "#5ac8c8", "#d3a7cb", "#c8a7cb", "#a6a7cb", "#81a7cb", "#5aa7c8", "#c986bc", "#c886bc", "#a686bc", "#8186bc", "#5a86bc", "#be64ac", "#be64ac", "#a664ac", "#8164ac", "#5a64ac"],
    'teal-red':    ["#e8e8e8", "#e2c3c3", "#dc9e9e", "#d67676", "#cf4848", "#bdd7dc", "#bdc3c3", "#bd9e9e", "#bd7676", "#bd4848", "#91c5cf", "#91c3c3", "#919e9e", "#917676", "#914848", "#63b2c3", "#63b2c3", "#639e9e", "#637676", "#634848", "#339fb5", "#339fb5", "#339e9e", "#337676", "#334848"],
    'purple-green': ["#e8e8e8", "#d0dec6", "#b8d3a3", "#9fc880", "#86bd5c", "#c9c5e6", "#c9c5c6", "#b8c5a3", "#9fc580", "#86bd5c", "#aba2e4", "#aba2c6", "#aba2a3", "#9fa280", "#86a25c", "#8c7ee2", "#8c7ec6", "#8c7ea3", "#8c7e80", "#867e5c", "#6c5ae0", "#6c5ac6", "#6c5aa3", "#6c5a80", "#6c5a5c"],
    'green-red': ["#d3d3d3", "#d0b3b2", "#ce9291", "#cb6d6b", "#c74240", "#bfd4b8", "#bdb49b", "#ba927e", "#b86e5d", "#b44337", "#a6d697", "#a4b680", "#a29368", "#a06f4d", "#9d432e", "#88d76d", "#86b65c", "#84944b", "#826f38", "#804421", "#47da16", "#46b912", "#45970f", "#44710b", "#434507"],
    'metabolic1' : ["#e8e8e8", "#dcbcc6", "#d18fa3", "#c45f7f", "#b52451", "#bddbc9", "#bdbcc6", "#bd8fa3", "#bd5f7f", "#b52451", "#91cda9", "#91bca9", "#918fa3", "#915f7f", "#912451", "#62bf87", "#62bc87", "#628f87", "#625f7f", "#622451", "#2eaf62", "#2eaf62", "#2e8f62", "#2e5f62", "#2e2451"],
    'metabolic2': ["#e8e8e8", "#dcbcc6", "#d18fa3", "#c45f7f", "#b52451", "#d8dbbf", "#d8bcbf", "#d18fa3", "#c45f7f", "#b52451", "#c8cd95", "#c8bc95", "#c88f95", "#c45f7f", "#b52451", "#b8c06b", "#b8bc6b", "#b88f6b", "#b85f6b", "#b52451", "#a8b240", "#a8b240", "#a88f40", "#a85f40", "#a82440"]
}
legend_colors = color_sets
legend_colors[conf['choropleth']][0:5], legend_colors[conf['choropleth']][5:10], legend_colors[conf['choropleth']][15:20], legend_colors[conf['choropleth']][20:25] = legend_colors[conf['choropleth']][20:25], legend_colors[conf['choropleth']][15:20], legend_colors[conf['choropleth']][5:10], legend_colors[conf['choropleth']][0:5]
    

In [21]:
def set_interval_value(x, break_1, break_2, break_3, break_4):
    if x <= break_1: 
        return 0
    elif break_1 < x <= break_2: 
        return 1
    elif break_2 < x <= break_3: 
        return 2
    elif break_3 < x <= break_4: 
        return 3
    else: 
        return 4

In [23]:
def prepare_df(df, x=conf['x'], y=conf['y'],conf = conf_defaults()):
    
    # Check if arguments match all requirements
    if df[x].shape[0] != df[y].shape[0]:
        raise ValueError('ERROR: The list of x and y coordinates must have the same length.')
    
    # Calculate break points at percentiles 33 and 66
    if conf['bmode'] == 'percentiles':
        x_breaks = np.percentile(df[x], [20, 40, 60, 80])
        y_breaks = np.percentile(df[y], [20, 40, 60, 80])
    elif conf['bmode'] == 'equal intervals':
        # Use customized break points from the configuration
        x_breaks = mapclassify.EqualInterval(df[x], k=5).bins
        y_breaks = mapclassify.EqualInterval(df[y], k=5).bins
    elif conf['bmode'] == 'custom':
        # Use customized break points from the configuration
        x_breaks = conf['custom_breaks_x']
        y_breaks = conf['custom_breaks_y']
    elif conf['bmode'] == 'ficher-jenks':
        x_breaks = mapclassify.FisherJenks(df[x], k=5).bins
        y_breaks = mapclassify.FisherJenks(df[y], k=5).bins
    elif conf['bmode'] == 'natural breaks':
        x_breaks = mapclassify.NaturalBreaks(df[x], k=5).bins
        y_breaks = mapclassify.NaturalBreaks(df[y], k=5).bins
    else:
        raise ValueError('ERROR: Invalid bivariate mode specified.')
        
    # Assign values of both variables to one of three bins (0, 1, 2, 3, 4)
    x_bins = [set_interval_value(value_x, x_breaks[0], x_breaks[1], x_breaks[2], x_breaks[3]) for value_x in df[x]]
    y_bins = [set_interval_value(value_y, y_breaks[0], y_breaks[1], y_breaks[2], y_breaks[3]) for value_y in df[y]]
    
    # Calculate the position of each x/y value pair in the 9-color matrix of bivariate colors
    df['biv_bins'] = [int(value_x + 5 * value_y) for value_x, value_y in zip(x_bins, y_bins)]
    
    #Assign colors
    color_sets = {
    'pink-blue':   ["#e8e8e8", "#c8e1e1", "#a6d9d9", "#81d1d1", "#5ac8c8", "#dec8d9", "#c8c8d9", "#a6c8d9", "#81c8d1", "#5ac8c8", "#d3a7cb", "#c8a7cb", "#a6a7cb", "#81a7cb", "#5aa7c8", "#c986bc", "#c886bc", "#a686bc", "#8186bc", "#5a86bc", "#be64ac", "#be64ac", "#a664ac", "#8164ac", "#5a64ac"],
    'teal-red':    ["#e8e8e8", "#e2c3c3", "#dc9e9e", "#d67676", "#cf4848", "#bdd7dc", "#bdc3c3", "#bd9e9e", "#bd7676", "#bd4848", "#91c5cf", "#91c3c3", "#919e9e", "#917676", "#914848", "#63b2c3", "#63b2c3", "#639e9e", "#637676", "#634848", "#339fb5", "#339fb5", "#339e9e", "#337676", "#334848"],
    'purple-green': ["#e8e8e8", "#d0dec6", "#b8d3a3", "#9fc880", "#86bd5c", "#c9c5e6", "#c9c5c6", "#b8c5a3", "#9fc580", "#86bd5c", "#aba2e4", "#aba2c6", "#aba2a3", "#9fa280", "#86a25c", "#8c7ee2", "#8c7ec6", "#8c7ea3", "#8c7e80", "#867e5c", "#6c5ae0", "#6c5ac6", "#6c5aa3", "#6c5a80", "#6c5a5c"],
    'green-red': ["#d3d3d3", "#d0b3b2", "#ce9291", "#cb6d6b", "#c74240", "#bfd4b8", "#bdb49b", "#ba927e", "#b86e5d", "#b44337", "#a6d697", "#a4b680", "#a29368", "#a06f4d", "#9d432e", "#88d76d", "#86b65c", "#84944b", "#826f38", "#804421", "#47da16", "#46b912", "#45970f", "#44710b", "#434507"],
    'metabolic1' : ["#e8e8e8", "#dcbcc6", "#d18fa3", "#c45f7f", "#b52451", "#bddbc9", "#bdbcc6", "#bd8fa3", "#bd5f7f", "#b52451", "#91cda9", "#91bca9", "#918fa3", "#915f7f", "#912451", "#62bf87", "#62bc87", "#628f87", "#625f7f", "#622451", "#2eaf62", "#2eaf62", "#2e8f62", "#2e5f62", "#2e2451"],
    'metabolic2': ["#e8e8e8", "#dcbcc6", "#d18fa3", "#c45f7f", "#b52451", "#d8dbbf", "#d8bcbf", "#d18fa3", "#c45f7f", "#b52451", "#c8cd95", "#c8bc95", "#c88f95", "#c45f7f", "#b52451", "#b8c06b", "#b8bc6b", "#b88f6b", "#b85f6b", "#b52451", "#a8b240", "#a8b240", "#a88f40", "#a85f40", "#a82440"]
    }
    for i in range(25):
        df.loc[df['biv_bins'] == i, 'colors'] = color_sets[conf['choropleth']][i]  
    
    return df

In [25]:
def create_bivariate_map(import_html=False,import_png=False):
   
    df = pd.read_csv(conf['csv'])
    df = prepare_df(df)
    gdf = gpd.read_file(conf['geo'])
    merged_gdf = gdf.merge(df, on=conf['ID'])
    merged_gdf.to_file('merged_data.geojson', driver='GeoJSON')
    geojson = 'merged_data.geojson'

    # calculate the center coordinates of the GeoJSON data
    bounds = merged_gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Define the tile layer URL template - default Carto DB Positron no labels
    tileset_url = 'https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png'
    attribution = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
    m = folium.Map(location=[center_lat, center_lon], tiles=tileset_url,attr=attribution, zoom_start=conf['map_zoom'], control_scale = True, zoom_control=True,scrollWheelZoom=True,dragging=True)
    folium.GeoJson(geojson, style_function=lambda x: {'fillColor': x['properties']['colors'], 'color': 'white', 'weight': 0.6, 'fillOpacity': 0.8},
                   tooltip=folium.features.GeoJsonTooltip(fields=[conf['id'],conf['x'],conf['y']], aliases=[['ID'],conf['legend_x'],conf['legend_y']])).add_to(m)
    #m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
    # create the legend html string
    font_style = "font-family: 'Roboto', sans-serif;"
    legend_html = '''
     <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 200px; height: 250px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; border-radius:5px;">
        <div style="display: flex; flex-direction: column; align-items: left; font-family: 'Roboto', sans-serif;">
            <div style="text-align: center; font-weight: bold;">Legend</div>
    
            <div style="display: flex; align-items: center;">
                <div style="background-color: #B3D1FF; margin-left: 10px; border: 1px solid #89A4D6; height: 10px; width: 10px;"></div>
                <span style="display: inline-block; margin-left: 10px;">Lake</span>
            </div>
    
            <div style="display: flex; align-items: center;">
                <div style="background-color: #B3D1FF; margin-left: 10px; height: 2px; width: 11px;"></div>
                <span style="display: inline-block; margin-left: 10px;">River</span>
            </div>
    
            <div style="display: flex; align-items: center;">
                <div style="background-color: black; margin-left: 10px; height: 10px; width: 10px; border-radius: 50%;"></div>
                <span style="display: inline-block; margin-left: 10px;">Major cities</span>
            </div>
    
            <div style="display: flex; align-items: center;">
                <div style="background-color: red; margin-left: 10px; height: 10px; width: 10px; border-radius: 50%;"></div>
                <span style="display: inline-block; margin-left: 10px;">Diageo site</span>
            </div>
        </div>

            <br>
            <table>
                <tr>
                    <td rowspan="5" style="padding-right:40px;vertical-align: middle;writing-mode: vertical-lr;transform: rotate(180deg);height: 100px; overflow-y: auto;white-space: normal;">{}🠒</td>
                        <div style="display: flex; flex-direction: column;">
                        <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                        <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                        <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                        <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                        <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                        
                </tr>
               
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                </tr>
                <tr>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                </tr>
                <tr>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                </tr>
                <tr>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                    <td><div style="background-color:{}; height: 20px; width: 20px;"></div></td>
                </tr>
                <tr>
                    <td></td>
                    <td colspan="5" style="text-align:left;padding-top:0px;">{} 🠒</td>
                </tr>
            </table>
        </div>
    '''.format(conf['legend_y'], *legend_colors[conf['choropleth']], conf['legend_x'])

    # create a div element to wrap the legend html and apply styles to it
    legend_div = folium.Element(
        "<div style='position: fixed; bottom: 20px; right: 20px; z-index: 1000; padding: 0px; border-radius: 0x;'>" +
        legend_html +
        "</div>"
    )
    
    #Points
    locations_geojson= "Locations.geojson"
    locations_data = gpd.read_file(locations_geojson)
    for _, location in locations_data.iterrows():
        folium.CircleMarker(
            location=[location.geometry.y, location.geometry.x],
            radius=2,
            color='red',
            fill=True,
            tooltip=location["Site name"]
        ).add_to(m)
   
    cities_geojson= "cities.geojson"
    cities_data = gpd.read_file(cities_geojson)
    for _, location in cities_data.iterrows():
        folium.CircleMarker(
            location=[location.geometry.y, location.geometry.x],
            radius=2,
            color='black',
            fill=True,
        ).add_to(m)
        
         # Add label
        folium.Marker(
            location=[location.geometry.y, location.geometry.x],
            icon=folium.DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 8pt; font-weight: bold;{font_style}">{location["NAME"]}</div>'
            )
        ).add_to(m)
        
    #Lakes
    lake_geojson = "lakes_mexico.geojson"
    lake_data = gpd.read_file(lake_geojson)
    folium.GeoJson(lake_data, style_function=lambda x: {'fillColor': '#B3D1FF','fillOpacity': 1,'color': '#89A4D6','weight': 0.8}).add_to(m)
    
    #rivers
    rivers_geojson = "rivers.geojson"
    rivers_data = gpd.read_file(rivers_geojson)
    folium.GeoJson(rivers_data, style_function=lambda x: {'fillOpacity': 1,'color': '#60adcd','weight':1}).add_to(m)  
       
    # add the legend div to the map
    m.get_root().html.add_child(legend_div)
    
    # add a minimap to the folium map
    minimap = plugins.MiniMap(toggle_display=False, position='bottomleft')
    m = m.add_child(minimap)
    

    arrow_html = '''
     <div style="position: fixed; 
             top: 25px; left: 25px; width: 60px; height: 90px; 
             border:0px solid grey; z-index:9999; font-size:7px;
             background-color:transparent;
             ">
    <svg height="50" width="50" style="transform: rotate(0deg); margin-left: 7.5px; margin-top: 12.5px;">
      <polygon points="25,0 0,50 25,35 50,50" style="fill:#36454F;stroke:black;stroke-width:0.5" />
    </svg>
    <br>
    
     </div>
     '''

    # Add the legend to the map
    m.get_root().html.add_child(folium.Element(arrow_html))


 
    
    if import_html:
        m.save("bivariate_map.html")  # Save as HTML

    if import_png:
        m.save("bivariate_map.jpg")  # Save as HTML
        
    return m

In [27]:
create_bivariate_map(import_html=False,import_png=False)